In [43]:
import os 
import sys
import numpy as np
import pandas as pd 
import pickle
from tqdm import tqdm


In [44]:
import torch
import torch.nn as nn
import torch.functional as F
from torch.utils.data import DataLoader

In [45]:
audio_data_path = '../11755-ISR-HW1/snf_7/'
video_data_path = '../11775-hw2-handout/data/cnn2d_1d/'

In [46]:
train_df = pd.read_csv('data/labels/train.csv')
val_df = pd.read_csv('data/labels/val.csv')
test_df = pd.read_csv('data/labels/test_for_students.csv')

In [47]:
audio_file_list = os.listdir(audio_data_path)
video_file_list = os.listdir(video_data_path)

In [48]:
feat_all = []
label_all = []


for ix, row in tqdm(train_df.iloc[:].iterrows()):
    # print(train_df.iloc[ix].Id)
    Id = train_df.iloc[ix].Id
    category = train_df.iloc[ix].Category
    if Id+'.csv' in audio_file_list:
        audio_feat = pd.read_csv(os.path.join(audio_data_path, Id+'.csv')).values
        audio_feat = audio_feat.reshape(-1, 1)
    else:
        print(Id)
        continue
    if Id+'.pkl' in video_file_list:
        with open(os.path.join(video_data_path, Id+'.pkl'), 'rb') as f:
            video_feat = pickle.load(f)
            video_feat = video_feat[1].numpy().reshape(-1, 1)
    else:
        print(Id)
        continue

    feature = np.concatenate([audio_feat, video_feat])
    feat_all.append(feature)
    label_all.append(category)
    with open('data/concat/{}.pkl'.format(Id), 'wb') as f:
        pickle.dump(feature, f)
feat_all = np.stack(feat_all)
print(feat_all.shape)

293it [00:07, 39.17it/s]

In [ ]:
config = {'batch_size': 64,
        'learning_rate': 0.001,
        }

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [ ]:
        feat_all = []
        label_all = []
        
        for ix, row in tqdm(train_df.iloc[:10].iterrows()):
            Id = train_df.iloc[ix].Id
            category = train_df.iloc[ix].Category
            if Id+'.csv' in audio_file_list:
                audio_feat = pd.read_csv(os.path.join(audio_data_path, Id+'.csv')).values
                audio_feat = audio_feat.reshape(-1, 1)
            else:
                print(Id)
                continue
            if Id+'.pkl' in video_file_list:
                with open(os.path.join(video_data_path, Id+'.pkl'), 'rb') as f:
                    video_feat = pickle.load(f)
                    video_feat = video_feat[1].numpy().reshape(-1, 1)
            else:
                print(Id)
                continue
            

            feature = np.concatenate([audio_feat, video_feat])
            feat_all.append(feature)
            label_all.append(category)

        feat_all = np.stack(feat_all)
        label_all = np.stack(label_all).reshape(-1, 1)

        length = len(feat_all)
        print(feat_all.shape)
        print(label_all.shape)
        print(length)

10it [00:00, 227.65it/s]

(10, 1535, 1)
(10, 1)
10


In [ ]:
class FusionDataset(torch.utils.data.Dataset):
    def __init__(self, audio_data_path, video_data_path, label_df, partition= "train", limit=-1):
        
        self.feat_all = []
        self.label_all = []
        
        for ix, row in tqdm(label_df.iloc[:].iterrows()):
            Id = label_df.iloc[ix].Id
            category = train_df.iloc[ix].Category

            if Id+'.csv' in audio_file_list:
                audio_feat = pd.read_csv(os.path.join(audio_data_path, Id+'.csv')).values
                audio_feat = audio_feat.reshape(-1, 1)
            else:
                print(Id)
                continue
            if Id+'.pkl' in video_file_list:
                with open(os.path.join(video_data_path, Id+'.pkl'), 'rb') as f:
                    video_feat = pickle.load(f)
                    video_feat = video_feat[1].numpy().reshape(-1, 1)
            else:
                print(Id)
                continue
            

            feature = np.concatenate([audio_feat, video_feat])
            self.feat_all.append(feature)
            self.label_all.append(category)

        self.feat_all = np.stack(self.feat_all)
        self.label_all = np.stack(self.label_all).reshape(-1, 1)

        self.length = len(self.feat_all)
        print(self.feat_all.shape)
        print(self.label_all.shape)
        
    
    def __len__(self):
            return self.length

    def __getitem__(self, ind):
        
        feature = self.feat_all[ind]
        feature = np.ndarray.flatten(feature) # TODO: Flatten to get 1d data

        feature = torch.FloatTensor(feature) # Convert to tensors
        label = torch.tensor(self.label_all[ind])  

        return feature, label

In [ ]:
train_loader = DataLoader(FusionDataset, num_workers= 4, batch_size=config['batch_size'], shuffle= False)

In [ ]:
class Network(torch.nn.Module):

    def __init__(self, dropout):

        super(Network, self).__init__()

        input_size = 1535 #Why is this the case? #375
        output_size = 15 #Why?

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 2048),
            torch.nn.ReLU(),    
            torch.nn.Dropout(dropout),
            torch.nn.Linear(2048, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(2048, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(2048, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(2048, 1024),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, output_size)
        )      

    def forward(self, x):
        out = self.model(x)

        return out

In [42]:
for ix, data in train_loader:
    print(ix, data)
    break

TypeError: object of type 'type' has no len()

In [23]:
model = Network(dropout=0).to(device)
data, label = next(iter(train_loader))
# Check number of parameters of your network - Remember, you are limited to 20 million parameters for HW1 (including ensembles)
# summary(model, frames.to(device))

TypeError: object of type 'type' has no len()